In [38]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
from pandas_datareader import data as pdr
from ta.trend import SMAIndicator

yf.pdr_override ()

#stock = input ("Enter a stock ticker symbol: ")
stock = "AMD"

#start = dt.datetime (2019, 1, 1)
#end = dt.datetime (2020, 1, 1)

start = dt.datetime.now() - dt.timedelta(days=300)
end = dt.datetime.now()

print(end)
df = pdr.get_data_yahoo (stock, start, end)


2021-08-05 12:13:45.462022
[*********************100%***********************]  1 of 1 completed


In [34]:
def get_avwap (df):
    df['avg price vol'] = ((df['High'] + df['Low'] + df['Adj Close'])/3) * df['Volume']
    df['avg price vol cum sum'] = df['avg price vol'].cumsum()
    df['vol cum sum'] = df['Volume'].cumsum()
    df['AWVAP'] = df['avg price vol cum sum'] / df['vol cum sum']
    return df.tail(1).iloc[0]['AWVAP']

In [42]:
from datetime import date

In [56]:
high_rec = df[df['High']==df['High'].max()] 
high_date = high_rec.index[0]   
delta = pd.Timestamp.today() - high_date

if delta.days < 6:
    print(delta.days)



1


In [19]:
high_date = rec.index[0]
df.loc[high_date]['High']


588.8400268554688